Data Analytics Starts from here

Importing Libraries

In [2]:
import pandas as pd
import json
import boto3
from io import StringIO, BytesIO

# Enable pandas options for better output readability
pd.set_option('display.max_columns', None)  
pd.set_option('display.width', 1000)


Load Datasets from S3
1. Time-Series Data (pr.data.0.Current)
2. Population Data (datausa_population.json)

Load Time-Series Data from S3 (pr.data.0.Current)

In [3]:
# AWS S3 Configuration
S3_BUCKET = "arc-cloud-dq"
s3 = boto3.client("s3")

# Define file paths
csv_key = "pr.data.0.Current"

# Fetch CSV from S3
response = s3.get_object(Bucket=S3_BUCKET, Key=csv_key)

# Read the CSV file into a DataFrame
bls_df = pd.read_csv(response["Body"], delimiter="\t", skipinitialspace=True)

# Display first few rows
print("🔹 BLS Time-Series Data (pr.data.0.Current):")
print(bls_df.head())


🔹 BLS Time-Series Data (pr.data.0.Current):
   series_id          year period  value footnote_codes
0  PRS30006011        1995    Q01    2.6            NaN
1  PRS30006011        1995    Q02    2.1            NaN
2  PRS30006011        1995    Q03    0.9            NaN
3  PRS30006011        1995    Q04    0.1            NaN
4  PRS30006011        1995    Q05    1.4            NaN


Trimming Whitespaces as per hints: skipinitialspace=True ensures leading/trailing spaces are removed.
Delimiter Issues: Tab-separated (\t) ensures correct parsing.
Column Name Cleanup:  refining this in the next step.

Load Population Data from S3 (datausa_population.json)

In [4]:
# Load JSON file from S3
json_key = "datausa_population.json"
response = s3.get_object(Bucket=S3_BUCKET, Key=json_key)

# Parse JSON into a dictionary
population_data = json.loads(response["Body"].read())

# Convert to Pandas DataFrame
pop_df = pd.DataFrame(population_data["data"])

# Rename columns for clarity
pop_df = pop_df.rename(columns={"Nation": "nation", "Population": "population", "Year": "year"})

# Convert year & population to integers
pop_df["year"] = pop_df["year"].astype(int)
pop_df["population"] = pop_df["population"].astype(int)

print("\n🔹 Population Data (datausa_population.json):")
print(pop_df.head())



🔹 Population Data (datausa_population.json):
  ID Nation         nation  ID Year  year  population    Slug Nation
0   01000US  United States     2022  2022   331097593  united-states
1   01000US  United States     2021  2021   329725481  united-states
2   01000US  United States     2020  2020   326569308  united-states
3   01000US  United States     2019  2019   324697795  united-states
4   01000US  United States     2018  2018   322903030  united-states


Parsing JSON Safely: Using json.loads().
Renaming Columns: More meaningful names (year, population).
Converting Data Types: Prevents errors in computations.

Data Cleaning & Standardization
Now, we clean both datasets to ensure smooth merging & filtering.

In [5]:
# Trim whitespace in column names
bls_df.columns = bls_df.columns.str.strip()

# Trim whitespace in all string columns
bls_df = bls_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Convert `year` column to integer
bls_df["year"] = pd.to_numeric(bls_df["year"], errors='coerce').astype("Int64")  # Handles NaN safely

# Convert `value` column to float
bls_df["value"] = pd.to_numeric(bls_df["value"], errors='coerce')

# Drop NaN rows if needed
bls_df = bls_df.dropna()

# Display cleaned data
print("✅ Cleaned BLS Time-Series Data:")
print(bls_df.head())


✅ Cleaned BLS Time-Series Data:
       series_id  year period    value footnote_codes
147  PRS30006011  2024    Q03   -0.500              R
447  PRS30006013  2024    Q03  103.852              R
597  PRS30006021  2024    Q03   -0.400              R
747  PRS30006022  2024    Q03   -1.700              R
897  PRS30006023  2024    Q03   97.062              R


Whitespace Removal: Ensures column names & data are clean.
Data Type Conversion: Prevents merging issues.
Handling Missing Values: dropna() removes problematic rows.

Compute Population Statistics (Mean & Std Dev) -- Find the mean and standard deviation of the US population from 2013 to 2018.

In [7]:
# Filter for years 2013-2018
pop_filtered_df = pop_df[(pop_df["year"] >= 2013) & (pop_df["year"] <= 2018)]

# Compute statistics
mean_population = pop_filtered_df["population"].mean()
std_population = pop_filtered_df["population"].std()

print(f"✅ Mean Population (2013-2018): {mean_population}")
print(f"✅ Standard Deviation of Population (2013-2018): {std_population}")


✅ Mean Population (2013-2018): 317437383.0
✅ Standard Deviation of Population (2013-2018): 4257089.5415293295


Find the Best Year for Each series_id
Rearc requires:

For each series_id, find the year with the highest sum of "value".

In [10]:
# Aggregate by `series_id` and `year`
best_year_df = bls_df.groupby(["series_id", "year"])["value"].sum().reset_index()

# Identify the best year for each series_id
best_year_df = best_year_df.loc[best_year_df.groupby("series_id")["value"].idxmax()]

# Rename column for clarity
best_year_df = best_year_df.rename(columns={"value": "max_annual_value"})

print("✅ Best Year Per Series ID:")
print(best_year_df.head())


✅ Best Year Per Series ID:
     series_id  year  max_annual_value
0  PRS30006011  2024            -0.500
1  PRS30006013  2024           103.852
2  PRS30006021  2024            -0.400
3  PRS30006022  2024            -1.700
4  PRS30006023  2024            97.062


Generate a Report for PRS30006032  -- Find the value for PRS30006032 in Q01 and match it with population.

In [12]:
# Filter for PRS30006032 and Q01
filtered_bls_df = bls_df[(bls_df["series_id"] == "PRS30006032") & (bls_df["period"] == "Q01")]

# Merge with Population Data
final_report = filtered_bls_df.merge(pop_df, on="year", how="left")[["series_id", "year", "period", "value", "population"]]

print("✅ Final Report:")
print(final_report)


✅ Final Report:
Empty DataFrame
Columns: [series_id, year, period, value, population]
Index: []


In [13]:
print(bls_df["series_id"].unique())

['PRS30006011' 'PRS30006013' 'PRS30006021' 'PRS30006022' 'PRS30006023'
 'PRS30006032' 'PRS30006033' 'PRS30006061' 'PRS30006062' 'PRS30006063'
 'PRS30006091' 'PRS30006092' 'PRS30006093' 'PRS30006101' 'PRS30006102'
 'PRS30006103' 'PRS30006112' 'PRS30006113' 'PRS30006151' 'PRS30006152'
 'PRS30006153' 'PRS30006162' 'PRS30006163' 'PRS30006211' 'PRS30006212'
 'PRS30006213' 'PRS31006022' 'PRS31006023' 'PRS31006032' 'PRS31006033'
 'PRS31006062' 'PRS31006063' 'PRS31006091' 'PRS31006092' 'PRS31006093'
 'PRS31006101' 'PRS31006102' 'PRS31006103' 'PRS31006112' 'PRS31006113'
 'PRS31006152' 'PRS31006153' 'PRS31006161' 'PRS31006162' 'PRS31006163'
 'PRS31006212' 'PRS31006213' 'PRS32006012' 'PRS32006013' 'PRS32006022'
 'PRS32006023' 'PRS32006032' 'PRS32006033' 'PRS32006061' 'PRS32006062'
 'PRS32006063' 'PRS32006091' 'PRS32006092' 'PRS32006093' 'PRS32006101'
 'PRS32006102' 'PRS32006103' 'PRS32006112' 'PRS32006113' 'PRS32006151'
 'PRS32006152' 'PRS32006153' 'PRS32006161' 'PRS32006162' 'PRS32006163'
 'PRS3

In [14]:
print(bls_df["period"].unique())

['Q03' 'Q02']
